## ❣️Author's Note
If you like this notebook, consider sharing it to others or give feedback. I would highly appreciate any comments, suggestions and recommendations to improve the notebook.

<div style="padding:20px;color:black;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#d9d9d9;overflow:hidden;font-weight:500">
Practice with Visualization & Analysis & XGB skills in DS</div>

# 📈Overview

    
**The tasks completed by this kernel are as follows:**

1. **clean and manipulate the data** with python and figure out the distribution of each features.
2. use matplotlib & seaborn to **visualization some features** which might be significant in build the model.
3. use **statistic knowledge**(hypothesis test, correspondence analysis) to analysis some features
4. use **XGB** Algorithm to **classificate** the dataset and analysis the result.

<h1 style="padding:10px;background-color:orange;margin:0;color:black;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Environment Setting</h1>
<br>

# 🛠️ Import Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import json
import sklearn
import re

#sklearn library
# 1.model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold

# 2.preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

# 3.metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# 4.model
import xgboost as xgb
from xgboost import plot_tree

import warnings
warnings.filterwarnings('ignore')

# 🛠️ Painting environment setting

In [ ]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
font = {'family':'Helvetica, Ariel',
        'weight':'normal',
        'size':12}
plt.rc('font', **font)
sns.set(rc={"figure.dpi": 300, 'savefig.dpi': 300})
sns.set_context('notebook')
sns.set_style("ticks")
FIG_FONT = dict(family="Helvetica, Ariel", weight="bold", color="#7f7f7f")
sns.set_palette('Spectral')

<h1 style="padding:10px;background-color:orange;margin:0;color:black;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">EDA</h1>
<br>

<div id='imports'
     style = "display: fill;
              border-radius: 10px;
              background-color: #6867AC;">
    <h2 style = "padding: 15px; 
                 color: White;
                 text-align: left;
                 font-family: Trebuchet MS;">Load the dataset
    </h2>
</div>

In [ ]:
space_titanic= pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
df=space_titanic.copy()
df.head()

# 📋 Have a quick glance at the dataset
use and .describe() to see the structure of the dataset and use .info() to find the statistical descriptions of the continuous features

In [ ]:
df.info()

In [ ]:
df.describe()

<h2 style="text-align: justify; font-size:18px; color:k">
use "profile_report" we can find the following characteristics in features:</h2>

1. Percentage of missing value in each feature
2. Distinct values in each categorized feature and frequency in each distinct value
3. Histogram in each categorized and continuous feature
4. Statistical describe, such as maximum, minimum, mean, etc, in each continuous feature
5. Correlations Matrix between each two features in the dataset


In [ ]:
import pandas_profiling
df.profile_report()

<div id='imports'
     style = "display: fill;
              border-radius: 10px;
              background-color: #6867AC;">
    <h2 style = "padding: 15px; 
                 color: White;
                 text-align: left;
                 font-family: Trebuchet MS;">Data Process
    </h2>
</div>

# 📋 Data processing
### The purpose of the data cleaning is:
1. Handle with missing values
2. Remove some outliers in numerical features
3. Standardized data format


In [ ]:
df=space_titanic.copy()
#1、Delete unnecessary columns 
df=df.drop(['Name','PassengerId'],axis=1)
    
#2、Handling null values 
df=df.dropna()
    
#3、create y value by using "Transported", convert to int type 标记y值，转换成int类型
target=df['Transported']
df=df.drop(['Transported'],axis=1)
target = target.astype(int)

#4、Processing category type data 
df['Cabin_1']=df['Cabin'].apply(lambda x: re.findall('[\w+]',x)[0] if pd.isnull(x) == False else float('nan'))
df['Cabin_2']=df['Cabin'].apply(lambda x: re.findall('[\w]+',x)[1] if pd.isnull(x) == False else float('nan'))
df['Cabin_2']=pd.cut(df['Cabin_2'].astype('int'),5,labels=[0,1,2,3,4])
df['Cabin_3']=df['Cabin'].apply(lambda x: re.findall('[\w]+',x)[2] if pd.isnull(x) == False else float('nan'))
df=df.drop(['Cabin'],axis=1)


#5、Obtaining continuous variable 
numaric_columns=list(df.select_dtypes(include=np.number).columns)
print("Numaric columns ("+str(len(numaric_columns))+") :",", ".join(numaric_columns))
    
#6、Obtaining discrete variables 
cat_columns=df.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns ("+str(len(cat_columns))+") :",", ".join(cat_columns))

#7、remove outliers
from collections import Counter
def detect_outliers(df, n, features_list):
    outlier_indices = [] 
    for feature in features_list: 
        Q1 = np.percentile(df[feature], 25)
        Q3 = np.percentile(df[feature], 75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR 
        outlier_list_col = df[(df[feature] < Q1 - outlier_step) | (df[feature] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col) 
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(key for key, value in outlier_indices.items() if value > n) 
    return multiple_outliers

outliers_to_drop = detect_outliers(df, 2, numaric_columns)
print("We will drop these {} indices: ".format(len(outliers_to_drop)), outliers_to_drop)



#8、Create the training and test datasets
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.2, 
                                                    random_state=100,stratify=target)
    
    
#9、Using continuous variable training data 
X_train_n=X_train[numaric_columns].copy()
X_test_n=X_test[numaric_columns].copy()
    

#10、Using discrete variable training data 
X_train_c=X_train[cat_columns]
X_test_c=X_test[cat_columns]
                               
#11、Ordinal label processing for discrete variables 
encoder=OrdinalEncoder()
X_train_c = encoder.fit_transform(X_train_c)
X_train_c=pd.DataFrame(X_train_c,index=X_train_n.index)
X_test_c = encoder.transform(X_test_c)
X_test_c=pd.DataFrame(X_test_c,index=X_test_n.index)
    
#12、Merge together
for i,column in enumerate(X_train_c.columns):
    X_train_n["cat_"+str(i+1)]=X_train_c[column]
    X_test_n["cat_"+str(i+1)]=X_test_c[column]

X_train = X_train_n
X_test = X_test_n

<h1 style="padding:10px;background-color:orange;margin:0;color:black;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Visualization</h1>
<br>

# 📋 Feature Visualization

In [ ]:
import scipy.stats as stats

#plot function
def univariate_double_plot(df=df, x=None, xlabel=None, explode=None,ylabel=None,palette=None,order=True,hue=None):
    sns.set_palette(palette)
    fig, ax = plt.subplots(1, 2, figsize=(20, 7))
    #count bar
    if order == True:
        feature_data = df[x].value_counts(ascending=True)
        sns.countplot(data=df, x=x, ax=ax[0], order=feature_data.index,hue = hue)
    
    else:
        feature_data = df[x].value_counts(sort=False).sort_index()
        sns.countplot(data=df, x=x, ax=ax[0],order=feature_data.index,hue = hue)
        
    #pie chart
    patches, texts, autotexts = ax[1].pie(feature_data.values,labels=feature_data.index,
                                      autopct='%.0f%%',textprops={'size': 20})
    for i in range(len(autotexts)):
        autotexts[i].set_color('white')

    #reduce non-data ink
    sns.despine(bottom=True, left=True)
    
    #set label
#     for i in range(len(feature_data.index)):
#         ax[0].text(i,feature_data.iloc[i]*0.9,feature_data.iloc[i],ha='center',
#                    fontsize=20,color='white')
    for i in range(len(ax[0].containers)):
        ax[0].bar_label(ax[0].containers[i], label_type='edge', size=12, padding=1, fontname="Helvetica, Ariel", 
                        color="#7f7f7f")
        
    ax[0].set_xlabel(xlabel=xlabel, size=12, fontdict=FIG_FONT)
    #ax[0].set_xticklabels(feature_data.index,rotation=20,fontsize = 'small')
    ax[0].set_ylabel(ylabel=ylabel)
    ax[1].set_ylabel(ylabel=ylabel)
    fig.text(0.5, 1, f'{xlabel} Distribution', size=16, fontdict=FIG_FONT, ha="center", va="center")
    plt.show()
    
def univariate_single_plot(df=df, x=None, xlabel=None, rotation=None,ylabel=None,palette=None):
    sns.set_palette(palette)
    fig, ax = plt.subplots(1, 1, figsize=(20, 7))
    feature_data = df[x].value_counts(ascending=True)
    sns.countplot(data=df, x=x, order=df[x].value_counts(ascending=True).index)
    sns.despine(bottom=True, left=True)
    plt.xlabel(xlabel=xlabel, size=14, fontdict=FIG_FONT)
    plt.xticks(rotation=rotation)
    plt.ylabel(ylabel=ylabel)
#     if bar_label:
#         ax.bar_label(ax.containers[0], label_type='edge', size=15, padding=1, fontname="Helvetica, Ariel", 
#                         color="k")
    for i in range(len(feature_data.index)):
        ax.text(i,feature_data.iloc[i]*0.9,feature_data.iloc[i],ha='center',
                   fontsize=20,color='white')
    plt.title(label=f'{xlabel} Distribution', size=18, fontdict=FIG_FONT)
    plt.show()
    
    
def univariate_numerical_plot(df=df, x=None, xlabel=None,ylabel=None,palette=None,bins=20):
    sns.set_palette(palette)
    fig, ax = plt.subplots(1, 3, figsize=(20, 7))
    #hist
    sns.histplot(bins=bins,data=df, x=x, kde=True, ax=ax[0])
    #box
    sns.boxplot(data=df, y=x, ax=ax[1])
    #prob
    plt.sca(ax[2])
    stats.probplot(df[x], dist = "norm", plot = plt)
    plt.ylabel('Variable quantiles')
    
    sns.despine(bottom=True, left=True)
    ax[0].set_xlabel(xlabel=xlabel, size=12, fontdict=FIG_FONT)
    ax[0].set_title(f'The histogram of {x}')
    ax[1].set_xlabel(xlabel=ylabel, size=12, fontdict=FIG_FONT)
    ax[0].set_ylabel(ylabel=ylabel,size=12, fontdict=FIG_FONT)
    ax[1].set_ylabel(ylabel=xlabel, size=12, fontdict=FIG_FONT)
    ax[1].set_title(f'The boxplot of {x}')
    fig.text(0.5, 1, f'{xlabel} Distribution', size=16, fontdict=FIG_FONT, ha="center", va="center")
    plt.show()

In [ ]:
%matplotlib inline
feature_data = univariate_numerical_plot(df,'Age')

In [ ]:
x='HomePlanet'
univariate_double_plot(space_titanic,x,x,hue='Transported')

In [ ]:
x='CryoSleep'
univariate_double_plot(space_titanic,x,x,hue='Transported')

In [ ]:
x='Destination'
univariate_double_plot(space_titanic,x,x,hue='Transported')

In [ ]:
x='VIP'
univariate_double_plot(space_titanic,x,x,hue='Transported')

<h1 style="padding:10px;background-color:orange;margin:0;color:black;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Data Analysis</h1>
<br>

# 📋 Correspondence Analysis
**Correspondence analysis between various features and Transported**

In [ ]:
def Chi_square_test(data,use_method=1,alpha=0.05):
    #1 Chi square independence test
    #Use mode 1 or mode 2: select 2 for 1 degree of freedom and 1 for others
    # use_method = 1 
    # alpha=0.05 # Set quantile value

    data=data.astype(int)
    data_j=data.sum()/data.sum().sum()  #Calculate column marginal probability
    data_i=data.sum(axis=1)             #Calculate row marginal probability

    #Here, the degree of freedom of the matrix should be:(columns_numbers-1)*(row_numbers-1)
    k=(len(data.columns)-1)*(len(data)-1)
    #The corresponding rejection domain is:
    rej_boundary=stats.chi2.ppf(1-alpha/2, k)

    data_exp=np.dot(data_i.values.reshape(-1,1),data_j.values.reshape(1,-1)) 
    data_exp=pd.DataFrame(data_exp,index=data_i.index,columns=data_j.index) 
    data_obj=data.values.flatten()
    data_exp=data_exp.values.flatten()

    def calculate_chi_val(use_method):
        if use_method == 1:
            # method1: The matrix is compressed into one-dimensional calculation,
            # which is applicable to the case where the degree of freedom is greater than 1
            # while using stats.chisquare, the degree of freedom is len(f_obs)-1-ddof=(len(data.columns)-1)*(len(data)-1)，so the parameter 'ddof' should be 
            chi_val,p_val=stats.chisquare(f_obs=data_obj,f_exp=data_exp,ddof=len(data_obj)-1-(len(data.columns)-1)*(len(data)-1))
            return chi_val,p_val

        elif use_method == 2:
            #method2：When the degree of freedom is 1, chi square test is performed on the frequency data, 
            # the probability will be low when the chi square distribution of continuous variables is used to calculate the probability
            #use this formula: X_c^2 = sum(|O-E|-0.5)^2/E
            chi_val=0
            for i in range(len(data_obj)):
                chi_val += (abs(data_obj[i]-data_exp[i])-0.5)**2/data_exp[i]
            p_val=scipy.stats.chi2.sf(abs(chi_val),k)
            return chi_val,p_val

    chi_val,p_val = calculate_chi_val(use_method)
    print('The Chi square value of the sample is:{:.3f}, The corresponding degrees of freedom is: {}, \nwhen alpha={}, p-value is :{:.8f}, Reject domain boundary is: {:.3f}'
          .format(chi_val,k,alpha,p_val,rej_boundary))

    if chi_val>rej_boundary:
        print(f'Conclusion： alpha=0.05, refuse H0, accpet H1. There are essential differences between {data.index.values} in {data.index.name}')
        print('-'*120)
    else:
        print(f'Conclusion: alpha=0.05, accpet H0, refuse H1. There is no essential difference between {data.index.values} in {data.index.name}')
        print('-'*120)

In [ ]:
categories=['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
for category in categories:
    feature=[category,'Transported']
    data = space_titanic[feature]
    data=data.pivot_table(index=feature[0],columns=feature[1],aggfunc=len,fill_value=0)
    if 'No internet service' in data.index.values:
        data.drop('No internet service',inplace=True)
    if 'No phone service' in data.index.values:
        data.drop('No phone service',inplace=True)
    print(f'data matrix:\n{data}\n')
    #print(f'The freedom of the matrix is {len(data.index.values)-1}\n')
    #print(data.index.values,data.index.name)
    Chi_square_test(data,len(data.index.values)-1,0.01)

<h1 style="padding:10px;background-color:orange;margin:0;color:black;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">XGB Classification</h1>
<br>

# 📋 XGBoost Classification
### XGBoost Classification & Hyperparameter selection

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
temp_dmatrix =xgb.DMatrix(data=X_train, label=y_train)

In [ ]:
PARAMETERS={"objective":'binary:logistic',"eval_metric":"auc","learning_rate": 0.5}
cv_results = xgb.cv(dtrain=temp_dmatrix, nfold=5,num_boost_round=10,params=PARAMETERS, as_pandas=True, seed=123 )
cv_results

In [ ]:
# max_depth
results=[]
for value in range(3,10,1):
    PARAMETERS['max_depth'] = value
    cv_results = xgb.cv(dtrain=temp_dmatrix, nfold=5,num_boost_round=10,params=PARAMETERS, as_pandas=True, seed=123)
    results.append((cv_results["train-auc-mean"].tail().values[-1],cv_results["test-auc-mean"].tail().values[-1]))
            
data = list(zip(range(3,10,1), results))
print(pd.DataFrame(data,columns=['max_depth',"auc(train,test)"]))

In [ ]:
# gamma
results=[]
for value in [0.1,0.2,0.5,1,1.5,2]:
    PARAMETERS['gamma'] = value
    cv_results = xgb.cv(dtrain=temp_dmatrix, nfold=5,num_boost_round=10,params=PARAMETERS, as_pandas=True, seed=123)
    results.append((cv_results["train-auc-mean"].tail().values[-1],cv_results["test-auc-mean"].tail().values[-1]))
            
data = list(zip([0.1,0.2,0.5,1,1.5,2], results))
print(pd.DataFrame(data,columns=['gamma',"auc(train,test)"]))

In [ ]:
# colsample_bytree
results=[]
for value in [.4,.5,.6,.7,.8,.9]:
    PARAMETERS['colsample_bytree'] = value
    cv_results = xgb.cv(dtrain=temp_dmatrix, nfold=5,num_boost_round=10,params=PARAMETERS, as_pandas=True, seed=123)
    results.append((cv_results["train-auc-mean"].tail().values[-1],cv_results["test-auc-mean"].tail().values[-1]))
            
data = list(zip([.4,.5,.6,.7,.8,.9], results))
print(pd.DataFrame(data,columns=['colsample_bytree',"auc(train,test)"]))

In [ ]:
learning_rate = [0.1,0.3,0.6]
max_depth = [3,6,8]
# min_child_weigh = list(range(0,5,2))
gamma=[0.2,0.5,1]
subsample = [.9,1]
colsample_bytree = [.8,1]
# scale_pos_weight = [.5,1,2]
# reg_alpha = np.linspace(0.001,1,2).tolist()
reg_lambda = np.linspace(0.001,1,2).tolist()
# n_estimators = [50,200,400]

parameters = { 'learning_rate': learning_rate,
              'max_depth': max_depth,
#              'min_child_weigh':min_child_weigh,
             'gamma':gamma,
             'subsample':subsample,
             'colsample_bytree':colsample_bytree,
#              'scale_pos_weight':scale_pos_weight,
#              'reg_alpha':reg_alpha,
             'reg_lambda':reg_lambda,
#              'n_estimators':n_estimators
             }

model = xgb.XGBClassifier(tree_method="gpu_hist")
clf = GridSearchCV(model, parameters,cv=3,scoring='accuracy',verbose=1,n_jobs=-1)
clf = clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)
roc_auc_score(y_test,y_test_pred)

In [ ]:
print(classification_report(y_test, y_test_pred, target_names=["0","1"]))

In [ ]:
from sklearn.metrics import plot_roc_curve
plot_roc_curve(clf, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import scikitplot as skplt

skplt.metrics.plot_confusion_matrix(y_train_pred,y_train,figsize=(5,5), cmap= 'YlGnBu',title='train data confusion matrix')
skplt.metrics.plot_confusion_matrix(y_test_pred,y_test,figsize=(5,5), cmap= 'YlGnBu',title='test data confusion matrix')

<div style="padding:20px;color:white;margin:20;font-size:270%;text-align:center;display:fill;border-radius:5px;background-color:#cc1100;overflow:hidden;font-weight:700">Please <b>UPVOTE</b> if it helped!</div>